In [1]:
import sys
sys.path.append('../src/')
import os
import math
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 12})

In [2]:
%load_ext autoreload
%autoreload 2

import utils

In [3]:
repo_path = '/cluster/home/eharve06/extrapolating-classifier-accuracy-to-bigger-datasets'
experiments_path = os.path.join(repo_path, 'experiments')
models_path = os.path.join(repo_path, 'models')

In [4]:
dataset_names = ['ChestX-ray14', 'Chest_X-Ray', 'BUSI', 'TMED-2', 'OASIS-3', 'Pilot']
forecasting = 'short_range'
labels = [['Atelectasis', 'Effusion', 'Infiltration'],
          ['Bacteria', 'Virus'],
          ['Normal', 'Benign', 'Malignant'],
          ['PLAX', 'PSAX', 'A4C', 'A2C'],
          ['Alzheimer’s'],
          ['WMD', 'CBI']]
models = ['PowerLaw', 'GPPowerLaw', 'Arctan', 'GPArctan']

row_index = -1
columns = ['Dataset', 'Label', 'Power law', 'GP power law (ours)', 'Arctan', 'GP arctan (ours)']
metrics_df = pd.DataFrame(columns=columns)

for dataset_index, dataset_name in enumerate(dataset_names):
    df = utils.load_experiment(os.path.join(experiments_path, '{}_{}.csv'.format(dataset_name, forecasting)))
    # Take mean of each random seed at each dataset size
    df = df.groupby('n').agg(lambda x: list(x))
    df.test_auroc = df.test_auroc.apply(lambda x: np.mean(x, axis=0))
    df.random_state = df.random_state.apply(lambda x: 'mean')
    df = df.reset_index()
    
    for label_index, label_name in enumerate(labels[dataset_index]):
        row_index = row_index+1
        metrics = [None]*len(models)
        # Plot data
        X_train, y_train, X_test, y_test = utils.split_df(df, index=label_index)
        # Load model
        for model_index, model_name in enumerate(models):
            model_filename = '{}_{}_{}.pt'.format(dataset_name, label_name, model_name)
            model_filepath = os.path.join(models_path, model_filename)
            model_objects = utils.load_model(model_name, model_filepath, X_train, y_train)
            metrics[model_index] = utils.print_metrics(model_objects, y_train, X_test, y_test, verbose=0)
        row = [dataset_name, label_name, metrics[0][0], metrics[1][0], metrics[2][0], metrics[3][0]]
        metrics_df.loc[row_index] = row
        print(metrics_df.loc[row_index])
    
rounded_df = metrics_df.round(3)
with open('rmse_short.tex', 'w') as f:
    f.write(rounded_df.to_latex(index=False))

Dataset                ChestX-ray14
Label                   Atelectasis
Power law                  0.343932
GP power law (ours)        0.328503
Arctan                     0.397446
GP arctan (ours)           0.396337
Name: 0, dtype: object
Dataset                ChestX-ray14
Label                      Effusion
Power law                  0.673123
GP power law (ours)        0.859261
Arctan                     0.670878
GP arctan (ours)            0.81119
Name: 1, dtype: object
Dataset                ChestX-ray14
Label                  Infiltration
Power law                  0.320141
GP power law (ours)        0.433044
Arctan                      2.26379
GP arctan (ours)           2.191777
Name: 2, dtype: object
Dataset                Chest_X-Ray
Label                     Bacteria
Power law                 0.211795
GP power law (ours)        0.22525
Arctan                    0.164136
GP arctan (ours)          0.168549
Name: 3, dtype: object
Dataset                Chest_X-Ray
Label          

/tmp/ipykernel_32406/1059431775.py:40: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(rounded_df.to_latex(index=False))


In [5]:
dataset_names = ['ChestX-ray14', 'Chest_X-Ray', 'TMED-2']
forecasting = 'long_range'
labels = [['Atelectasis', 'Effusion', 'Infiltration'],
          ['Bacteria', 'Virus'],
          ['PLAX', 'PSAX', 'A4C', 'A2C']]
models = ['PowerLaw', 'GPPowerLaw', 'Arctan', 'GPArctan']

row_index = -1
columns = ['Dataset', 'Label', 'Power law', 'GP power law (ours)', 'Arctan', 'GP arctan (ours)']
metrics_df = pd.DataFrame(columns=columns)

for dataset_index, dataset_name in enumerate(dataset_names):
    df = utils.load_experiment(os.path.join(experiments_path, '{}_{}.csv'.format(dataset_name, forecasting)))
    # Take mean of each random seed at each dataset size
    df = df.groupby('n').agg(lambda x: list(x))
    df.test_auroc = df.test_auroc.apply(lambda x: np.mean(x, axis=0))
    df.random_state = df.random_state.apply(lambda x: 'mean')
    df = df.reset_index()
    
    for label_index, label_name in enumerate(labels[dataset_index]):
        row_index = row_index+1
        metrics = [None]*len(models)
        # Plot data
        X_train, y_train, X_test, y_test = utils.split_df(df, index=label_index)
        # Load model
        for model_index, model_name in enumerate(models):
            model_filename = '{}_{}_{}.pt'.format(dataset_name, label_name, model_name)
            model_filepath = os.path.join(models_path, model_filename)
            model_objects = utils.load_model(model_name, model_filepath, X_train, y_train)
            metrics[model_index] = utils.print_metrics(model_objects, y_train, X_test, y_test, verbose=0)
        row = [dataset_name, label_name, metrics[0][0], metrics[1][0], metrics[2][0], metrics[3][0]]
        metrics_df.loc[row_index] = row
        print(metrics_df.loc[row_index])
    
rounded_df = metrics_df.round(3)
with open('rmse_long.tex', 'w') as f:
    f.write(rounded_df.to_latex(index=False))

Dataset                ChestX-ray14
Label                   Atelectasis
Power law                  1.854585
GP power law (ours)        1.721867
Arctan                     2.948669
GP arctan (ours)           2.946587
Name: 0, dtype: object
Dataset                ChestX-ray14
Label                      Effusion
Power law                  3.970832
GP power law (ours)        3.505888
Arctan                       3.9656
GP arctan (ours)           3.513107
Name: 1, dtype: object
Dataset                ChestX-ray14
Label                  Infiltration
Power law                  1.616261
GP power law (ours)        2.078804
Arctan                    15.349708
GP arctan (ours)          13.469627
Name: 2, dtype: object
Dataset                Chest_X-Ray
Label                     Bacteria
Power law                 1.067364
GP power law (ours)       1.088963
Arctan                    0.207979
GP arctan (ours)          0.210053
Name: 3, dtype: object
Dataset                Chest_X-Ray
Label          

/tmp/ipykernel_32406/60708135.py:37: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(rounded_df.to_latex(index=False))


In [6]:
dataset_names = ['ChestX-ray14', 'Chest_X-Ray', 'BUSI', 'TMED-2', 'OASIS-3', 'Pilot']
labels = [['Atelectasis', 'Effusion', 'Infiltration'],
          ['Bacteria', 'Virus'],
          ['Normal', 'Benign', 'Malignant'],
          ['PLAX', 'PSAX', 'A4C', 'A2C'],
          ['Alzheimer’s'],
          ['WMD', 'CBI']]
models = ['GPPowerLaw', 'GPArctan']

row_index = -1
columns = ['Dataset', 'Label', 'Power law', 'GP power law (ours)', 'Arctan', 'GP arctan (ours)']
metrics_df = pd.DataFrame(columns=columns)

for dataset_index, dataset_name in enumerate(dataset_names):
    for label_index, label_name in enumerate(labels[dataset_index]):
        row_index = row_index+1
        metrics = [None]*(2*len(models))
        
        df = utils.load_experiment(os.path.join(experiments_path, '{}_short_range.csv'.format(dataset_name)))
        # Take mean of each random seed at each dataset size
        df = df.groupby('n').agg(lambda x: list(x))
        df.test_auroc = df.test_auroc.apply(lambda x: np.mean(x, axis=0))
        df.random_state = df.random_state.apply(lambda x: 'mean')
        df = df.reset_index()
        
        # Plot data
        X_train, y_train, X_test, y_test = utils.split_df(df, index=label_index)
        # Load model
        for model_index, model_name in enumerate(models):
            model_filename = '{}_{}_{}.pt'.format(dataset_name, label_name, model_name)
            model_filepath = os.path.join(models_path, model_filename)
            model_objects = utils.load_model(model_name, model_filepath, X_train, y_train)
            metrics[model_index] = utils.print_metrics(model_objects, y_train, X_test, y_test, verbose=0)
            
        if dataset_name in ['ChestX-ray14', 'Chest_X-Ray', 'TMED-2']:
            
            df = utils.load_experiment(os.path.join(experiments_path, '{}_long_range.csv'.format(dataset_name)))
            # Take mean of each random seed at each dataset size
            df = df.groupby('n').agg(lambda x: list(x))
            df.test_auroc = df.test_auroc.apply(lambda x: np.mean(x, axis=0))
            df.random_state = df.random_state.apply(lambda x: 'mean')
            df = df.reset_index()

            # Plot data
            X_train, y_train, X_test, y_test = utils.split_df(df, index=label_index)
            # Load model
            for model_index, model_name in enumerate(models):
                model_filename = '{}_{}_{}.pt'.format(dataset_name, label_name, model_name)
                model_filepath = os.path.join(models_path, model_filename)
                model_objects = utils.load_model(model_name, model_filepath, X_train, y_train)
                metrics[2+model_index] = utils.print_metrics(model_objects, y_train, X_test, y_test, verbose=0)
        
            row = [dataset_name, label_name, metrics[0][4], metrics[1][4], metrics[2][4], metrics[3][4]]
            metrics_df.loc[row_index] = row
            print(metrics_df.loc[row_index])
    
        else:
            row = [dataset_name, label_name, metrics[0][4], metrics[1][4], None, None]
            metrics_df.loc[row_index] = row
            print(metrics_df.loc[row_index])
            
rounded_df = metrics_df.round(2)
with open('point_coverage.tex', 'w') as f:
    f.write(rounded_df.to_latex(index=False))

Dataset                ChestX-ray14
Label                   Atelectasis
Power law                     100.0
GP power law (ours)           100.0
Arctan                        100.0
GP arctan (ours)              100.0
Name: 0, dtype: object
Dataset                ChestX-ray14
Label                      Effusion
Power law                     100.0
GP power law (ours)           100.0
Arctan                         60.0
GP arctan (ours)               60.0
Name: 1, dtype: object
Dataset                ChestX-ray14
Label                  Infiltration
Power law                     100.0
GP power law (ours)           100.0
Arctan                        100.0
GP arctan (ours)                0.0
Name: 2, dtype: object
Dataset                Chest_X-Ray
Label                     Bacteria
Power law                    100.0
GP power law (ours)          100.0
Arctan                       100.0
GP arctan (ours)             100.0
Name: 3, dtype: object
Dataset                Chest_X-Ray
Label          

/tmp/ipykernel_32406/1653935397.py:64: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  f.write(rounded_df.to_latex(index=False))
